In [1]:
#import packages needed to run the notebook
import numpy
import skimage
import skimage.io
import pandas
import scipy.ndimage
import seaborn
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib
import os
import math
import matplotlib.patches as mpl_patches
from matplotlib.offsetbox import AnchoredText
from scipy import signal
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import UnivariateSpline
from collections import defaultdict
from bisect import bisect_left
from bisect import bisect_right
from statsmodels.stats import weightstats

In [ ]:
#Folder where movies, csv and label image are saved
input_folder = '/Volumes/BAGGINS/TRD2_test/input'
#Folder w
#Folder where graphs and created files will be stored
output_folder = '/Volumes/BAGGINS/TRD2_test/event_graph'

In [3]:
#Parameters that can be adjusted 

crop_size_begining = 100 #number of frames at the beginning that should be omitted
crop_size_end = 100 #number of frames at the beginning that should be omitted
max_event_frame = 15 #number of maximum event present at a single frame 
event_ext = 800 #number of frame to extend the event for peak and baseline detection
pixel_size = 5 #pixel size of the events 
event_min_intensity = 15
baseline_size = 200 #approximate frame number used for baseline and post event calculation

#for noisy images fluorescence intensity can change from frame to frame applying a 
#smooth filter to the data for correct peak detecion the smooth size also depends on the lenght of the event

smooth_long = 100000 #smooth size applied to long event for peak detection
smooth_mid = 150000 #smooth size applied to mid length event for peak detection
smooth_short = 50000 #smooth size applied to short events for peak detection

In [4]:
def find_lt(a, x):
    'Find rightmost value less than x'
    i = bisect_left(a, x)
    if i:
        return a[i-1]
    if i == 0:
        return x
    
def find_gt(a, x):
    'Find leftmost value greater than x'
    i = bisect_right(a, x)
    if i != len(a):
        return a[i]
    if i == len(a):
        return x

def find_btw(a, x, y):
    'Find rightmost value less than x'
    i = bisect_right(a, x)
    e = bisect_right(a, y)
    o = e - i
    return o-1

In [ ]:
file_list = os.listdir(input_folder)
for file in file_list:
    if file[-4:]==".tif": #use raw movie names
        movie = file.split(".")[0]
        
        #track_csv is the csv of the x,y,z position of the detected events
        tracks_csv = pandas.read_csv(os.path.join(input_folder,movie+'.csv')) #change the path to 
        
        #label_image is the image sequence with event anotation
        label_image = skimage.io.imread(os.path.join(input_folder,movie+'_lblImg_whole.tif'))
        
        #raw_image is the raw movie
        raw_image = skimage.io.imread(os.path.join(input_folder,movie+'.tif'))
        length = raw_image.shape[0]
        
        #Template to save track graphs
        allPDF=PdfPages(os.path.join(output_folder,movie+'_all.pdf'))
        devPDF=PdfPages(os.path.join(output_folder,movie+'_dev.pdf'))
        tracePDF=PdfPages(os.path.join(output_folder,movie+'_events.pdf'))
        
        #sort event ROI based on event number and frame number
        tracks_csv=tracks_csv.where(tracks_csv.notnull(), None)
        tracks_csv = tracks_csv.sort_values(by=['TRACK_ID', 'FRAME'])
        tracks_csv['TRACK_ID'] = tracks_csv['TRACK_ID'] + 1
        tracks_csv['FRAME'] = tracks_csv['FRAME'] + 1

        #filter events around cell movements
        tracks = tracks_csv.groupby("FRAME").agg(**{"TRACK_COUNT": pandas.NamedAgg(column='TRACK_ID', aggfunc="nunique"),
                                  "TRACK_LIST": pandas.NamedAgg(column='TRACK_ID', aggfunc="unique")})
      
        #tracks_lenght = tracks_csv.groupby("TRACK_ID").agg(**{"FRAME_COUNT": pandas.NamedAgg(column='FRAME', aggfunc="nunique")})

        multi_tracks = tracks.drop(tracks[tracks.TRACK_COUNT <= max_event_frame ].index)
        multi_tracks = multi_tracks.explode('TRACK_LIST')
        multi_tracks_list = multi_tracks.TRACK_LIST.unique()
        bad_tracks = multi_tracks_list.tolist()

        
        #save the x,y position at the start and end of the track for each event
        track_dic = {}
        for index,row in tracks_csv.iterrows():
            if row["TRACK_ID"] != None:
                if row["TRACK_ID"] not in bad_tracks:
                    track_id = int(row["TRACK_ID"])
                    if track_id not in list(track_dic.keys()):
                        track_dic[track_id] = {}
                        track_dic[track_id]["start_frame"] = track_dic[track_id]["stop_frame"] = row["FRAME"]
                        track_dic[track_id]["start_x"] = track_dic[track_id]["stop_x"] = row["POSITION_X"]
                        track_dic[track_id]["start_y"] = track_dic[track_id]["stop_y"] = row["POSITION_Y"]
                    else:
                        track_dic[track_id]["stop_frame"] = row["FRAME"]
                        track_dic[track_id]["stop_x"] = row["POSITION_X"]
                        track_dic[track_id]["stop_y"] = row["POSITION_Y"]
        track_list=list(track_dic.keys())

        #filter tracks that are just 1 frame
        for track in track_list:
            if int(track_dic[track]["stop_frame"])-int(track_dic[track]["start_frame"]) < 1:
                del track_dic[track]
        track_list=list(track_dic.keys())

        #filter tracks that started in the first crop_size_begining frames 
        for track in track_list:
            if int(track_dic[track]["start_frame"]) < crop_size_begining:
                del track_dic[track]
        track_list=list(track_dic.keys())

        #filter tracks that ends in the last crop_size_end frames 
        for track in track_list:
            if int(track_dic[track]["stop_frame"]) > int(length - crop_size_end):
                del track_dic[track]
        track_list=list(track_dic.keys())

        #Add ROI in the same x,y position at the start and end of the track even_ext number of frames
        for track in track_list:
            start = int(track_dic[track]["start_frame"])
            stop = int(track_dic[track]["stop_frame"])
            for x in range(1, event_ext):
                if start-x > 0:
                    label_image[start-x] = numpy.where(label_image[start] == track, track, label_image[start-x])
            for x in range(1, event_ext):
                if stop+x < length:
                    label_image[stop+x] = numpy.where(label_image[stop] == track, track, label_image[stop+x])

        #Add ROI in the x,y position in the missing frame where the previous frame ROI was
        for track in track_list:
            start = int(track_dic[track]["start_frame"])-event_ext
            stop = int(track_dic[track]["stop_frame"])+event_ext+1
            if start <= 0:
                start = 1
            else:
                start = start
            if stop > length:
                stop = length
            else:
                stop = stop

            for frame in range(start,stop,1):
                if track not in label_image[frame]:
                    label_image[frame] = numpy.where(label_image[frame-1] == track, track, label_image[frame])

        #Dilate ROI to pixel_size of event 
        for frame in range(length):
            A = label_image[frame]
            B = scipy.ndimage.maximum_filter(A, pixel_size)
            B[A != 0] = A[A != 0]
            label_image[frame] = B
        
        
        #Save measurement of event pre filtering                
        track_df = pandas.DataFrame(index=track_list)
        for frame in range(length):
            track_df[frame]=scipy.ndimage.mean(raw_image[frame],labels=label_image[frame],index=track_list)
        track_df=track_df.T
        track_df["timepoint"]=track_df.index        
        track_df.dropna(axis='columns', how='all', inplace=True)        
        track_df.to_csv(os.path.join(output_folder,'Beforefilter'+movie+'.csv'))

        #find inflection points in the second derivative to determine start and end of event and post-event and baseline
        infl_dic = {}
        track_list = []
        for track in track_df.columns:
            if track != "timepoint":
                track_list.append(track)
                start = int(track_dic[track]["start_frame"])
                stop = int(track_dic[track]["stop_frame"])
                lenght = stop - start
                infl_dic[track] = {}
                if lenght >= 100:
                    roi_position = '\n'.join((r'$\mathrm{x}=%.2f$' % float(track_dic[track]["start_x"]),
                                              r'$\mathrm{y}=%.2f$' % float(track_dic[track]["start_y"]),
                                              r'$\mathrm{start}=%.2f$' % float(track_dic[track]["start_frame"]),
                                              r'$\mathrm{Event_length}=%s$' % str(lenght)))
                    raw = track_df[track]
                    raw = raw.dropna()
                    t_raw = raw.index
                    mean = int(raw.values.mean())
                    spl = UnivariateSpline(t_raw, raw, k=4, s=int(smooth_long))
                    track_line = seaborn.lineplot(data=track_df, x="timepoint", y=track, linewidth=0.25,)
                    plt.plot(t_raw, spl(t_raw))
                    spl_d = spl.derivative(n=1)
                    plt.plot(t_raw, (spl_d(t_raw) / numpy.max(spl_d(t_raw)))+ mean)
                    infls = numpy.where(numpy.diff(numpy.sign(spl_d(t_raw))))[0]
                    max_value = (t_raw[numpy.where((spl(t_raw)==numpy.max(spl(t_raw))))[0]]).tolist()
                    t_infls = []
                    for i in infls:
                        if i != 0:
                            t_infl = t_raw[i]
                            t_infls.append(t_raw[i])
                            plt.axvline(x=t_infl, linewidth=0.25, ls='dotted', color='k')
                    track_line.axvline(x = int(track_dic[track]["start_frame"]), linewidth=0.25, ls='--', color='r')
                    track_line.axvline(x = int(track_dic[track]["stop_frame"]), linewidth=0.25, ls='--', color='b')
                    anchored_text = AnchoredText(roi_position, loc=2)
                    track_line.add_artist(anchored_text)
                    plt.savefig(devPDF,format='pdf')
                    plt.close()
                    infl_dic[track]['inflections']=t_infls
                    infl_dic[track]['point']=max_value[0]

                elif lenght > 50 and lenght < 100:
                    roi_position = '\n'.join((r'$\mathrm{x}=%.2f$' % float(track_dic[track]["start_x"]),
                                              r'$\mathrm{y}=%.2f$' % float(track_dic[track]["start_y"]),
                                              r'$\mathrm{start}=%.2f$' % float(track_dic[track]["start_frame"]),
                                              r'$\mathrm{Event_length}=%s$' % str(lenght)))
                    raw = track_df[track]
                    raw = raw.dropna()
                    t_raw = raw.index
                    mean = int(raw.values.mean())
                    spl = UnivariateSpline(t_raw, raw, k=4, s=int(smooth_mid))
                    track_line = seaborn.lineplot(data=track_df, x="timepoint", y=track, linewidth=0.25,)
                    plt.plot(t_raw, spl(t_raw))
                    spl_d = spl.derivative(n=1)
                    plt.plot(t_raw, (spl_d(t_raw) / numpy.max(spl_d(t_raw)))+mean)
                    infls = numpy.where(numpy.diff(numpy.sign(spl_d(t_raw))))[0]
                    max_value = (t_raw[numpy.where((spl(t_raw)==numpy.max(spl(t_raw))))[0]]).tolist()
                    t_infls = []
                    for i in infls:
                        if i != 0:
                            t_infl = t_raw[i]
                            t_infls.append(t_raw[i])
                            plt.axvline(x=t_infl, linewidth=0.25, ls='dotted', color='k')
                    track_line.axvline(x = int(track_dic[track]["start_frame"]), linewidth=0.25, ls='--', color='r')
                    track_line.axvline(x = int(track_dic[track]["stop_frame"]), linewidth=0.25, ls='--', color='b')
                    anchored_text = AnchoredText(roi_position, loc=2)
                    track_line.add_artist(anchored_text)
                    plt.savefig(devPDF,format='pdf')
                    plt.close()
                    infl_dic[track]['inflections']=t_infls
                    infl_dic[track]['point']=max_value[0]

                elif lenght <= 50:
                    roi_position = '\n'.join((r'$\mathrm{x}=%.2f$' % float(track_dic[track]["start_x"]),
                                              r'$\mathrm{y}=%.2f$' % float(track_dic[track]["start_y"]),
                                              r'$\mathrm{start}=%.2f$' % float(track_dic[track]["start_frame"]),
                                              r'$\mathrm{Event_length}=%s$' % str(lenght)))
                    raw = track_df[track]
                    raw = raw.dropna()
                    t_raw = raw.index
                    mean = int(raw.values.mean())
                    spl = UnivariateSpline(t_raw, raw, k=4, s=int(smooth_short))
                    track_line = seaborn.lineplot(data=track_df, x="timepoint", y=track, linewidth=0.25,)
                    plt.plot(t_raw, spl(t_raw))
                    spl_d = spl.derivative(n=1)
                    plt.plot(t_raw, (spl_d(t_raw) / numpy.max(spl_d(t_raw)))+mean)
                    infls = numpy.where(numpy.diff(numpy.sign(spl_d(t_raw))))[0]
                    max_value = (t_raw[numpy.where((spl(t_raw)==numpy.max(spl(t_raw))))[0]]).tolist()
                    t_infls = []
                    for i in infls:
                        if i != 0:
                            t_infl = t_raw[i]
                            t_infls.append(t_raw[i])
                            plt.axvline(x=t_infl, linewidth=0.25, ls='dotted', color='k')
                    track_line.axvline(x = int(track_dic[track]["start_frame"]), linewidth=0.25, ls='--', color='r')
                    track_line.axvline(x = int(track_dic[track]["stop_frame"]), linewidth=0.25, ls='--', color='b')
                    anchored_text = AnchoredText(roi_position, loc=2)
                    track_line.add_artist(anchored_text)
                    plt.savefig(devPDF,format='pdf')
                    plt.close()
                    infl_dic[track]['inflections']=t_infls
                    infl_dic[track]['point']=max_value[0]
        devPDF.close()
       
        for track in track_list:
            start = int(track_dic[track]["start_frame"])
            stop = int(track_dic[track]["stop_frame"])
            lenght = stop - start 
            infls = infl_dic[track]['inflections']
            peak = infl_dic[track]['point']
            track_dic[track]["new_start_frame"] =(find_lt(infls, start))
            track_dic[track]["new_stop_frame"] =(find_gt(infls, stop))
        
        #identify false events 
        false_event=['timepoint']
        for track in track_list:
            if track != "timepoint":
                if track not in false_event:
                    start_b = int(track_dic[track]["new_start_frame"])-baseline_size
                    stop_b = int(track_dic[track]["new_start_frame"])-1
                    start_e = int(track_dic[track]["new_start_frame"])
                    stop_e = int(track_dic[track]["new_stop_frame"])
                    start_ae = int(track_dic[track]["new_stop_frame"])+1
                    stop_ae = int(track_dic[track]["new_stop_frame"])+baseline_size
                    event = track_df.loc[start_e:stop_e, track]
                    baseline = track_df.loc[start_b:stop_b, track]
                    afterevent = track_df.loc[start_ae:stop_ae, track]
                    event = event.values
                    baseline = baseline.values
                    afterevent = afterevent.values

                    if event.max() - event.mean() <= event_min_intensity:
                        false_event.append(track) 

                    if event.max() <= baseline.max():
                        false_event.append(track) 

                    if event.max() <= afterevent.max():
                        false_event.append(track) 

        good_tracks = [track for track in track_list if track not in false_event]
        repeat_event=[]
        for track in good_tracks:
            if track not in repeat_event:
                now_track = track
                start_a = int(track_dic[track]["new_start_frame"])
                x_pos_a = float(track_dic[track]["start_x"])
                y_pos_a = float(track_dic[track]["start_y"])
                for track in good_tracks:
                    if track != now_track:
                        start_b = int(track_dic[track]["new_start_frame"])
                        x_pos_b = float(track_dic[track]["start_x"])
                        y_pos_b = float(track_dic[track]["start_y"])
                        start_dif = start_a - start_b
                        x_dif = x_pos_a - x_pos_b
                        y_dif = y_pos_a - y_pos_b
                        if -25 <= start_dif <= 25:
                            if -1 <= x_dif <= 1:
                                if -1 <= x_dif <= 1:
                                    repeat_event.append(track)
        good_tracks = [track for track in good_tracks if track not in repeat_event]

        #Determine vesicle type
        for track in good_tracks:
            if track != "timepoint":
                if track not in false_event:
                    start_b = int(track_dic[track]["new_start_frame"])-baseline_size
                    stop_b = int(track_dic[track]["new_start_frame"])-1
                    start_ae = int(track_dic[track]["new_stop_frame"])+1
                    stop_ae = int(track_dic[track]["new_stop_frame"])+baseline_size
                    baseline = track_df.loc[start_b:stop_b, track]
                    afterevent = track_df.loc[start_ae:stop_ae, track]
                    if weightstats.ttest_ind(baseline,afterevent)[1] < 0.05:
                        track_dic[track]["ves_type"] = 'Docked'
                    else:
                        track_dic[track]["ves_type"] = 'New Arrival'

        #save average fluorescence intensity of each frame for each ROI of the corresponding movie as a csv
        track_df.to_csv(os.path.join(output_folder,'TimeTrace(s)_'+movie+'.csv'), columns = good_tracks)

        #save Slice number information for each ROI startidx,pre_endidx,Slice,post_startidx,endidx,spike_start,spike_end
        track_dict = pandas.DataFrame.from_dict(track_dic, orient='index')
        track_dict.drop(['start_frame', 'stop_frame', 'stop_x', 'stop_y'], axis='columns', inplace=True)
        track_dict = track_dict.filter(items = good_tracks, axis=0)
        track_dict.rename(columns = {'new_start_frame':'spike_start', 'new_stop_frame':'spike_end',
                                     'start_x':'x', 'start_y':'y' }, inplace = True)
        track_df.to_csv(os.path.join(output_folder,'Measurement_'+movie+'.csv'))

        #graph an save a pdf of the graph of each filtered event 
        for track in track_df.columns:
            if track in good_tracks: 
                roi_position = '\n'.join((r'$\mathrm{x}=%.2f$' % float(track_dic[track]["start_x"]),
                                          r'$\mathrm{y}=%.2f$' % float(track_dic[track]["start_y"]),
                                          r'$\mathrm{start}=%.2f$' % float(track_dic[track]["new_start_frame"]), 
                                          r'$\mathrm{VesType}=%s$' % str(track_dic[track]["ves_type"])))
                track_line = seaborn.lineplot(data=track_df, x="timepoint", y=track, linewidth=0.25,)
                track_line.axvline(x = int(track_dic[track]["new_start_frame"]), linewidth=0.25, ls='--', color='r')
                track_line.axvline(x = int(track_dic[track]["new_stop_frame"]), linewidth=0.25, ls='--', color='b')
                anchored_text = AnchoredText(roi_position, loc=2)
                track_line.add_artist(anchored_text)
                plt.savefig(tracePDF,format='pdf')
                plt.close()
        tracePDF.close()

        #graph an save a pdf of the graph for all events
        for track in track_df.columns:
            if track in track_list: 
                if row["TRACK_ID"] != 'None':
                    roi_position = '\n'.join((r'$\mathrm{x}=%.2f$' % float(track_dic[track]["start_x"]),
                                              r'$\mathrm{y}=%.2f$' % float(track_dic[track]["start_y"]),
                                              r'$\mathrm{start}=%.2f$' % float(track_dic[track]["start_frame"])))
                    track_line = seaborn.lineplot(data=track_df, x="timepoint", y=track, linewidth=0.25,)
                    track_line.axvline(x = int(track_dic[track]["start_frame"]), linewidth=0.25, ls='--', color='r')
                    track_line.axvline(x = int(track_dic[track]["stop_frame"]), linewidth=0.25, ls='--', color='b')
                    anchored_text = AnchoredText(roi_position, loc=2)
                    track_line.add_artist(anchored_text)
                    plt.savefig(allPDF,format='pdf')
                    plt.close()
        allPDF.close()

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/BAGGINS/TRD2_test/output/Stream_img1_lblImg_whole.tif'